# Analysis of a multi-channel FISH dataset
Last update: 2022-03-11<br>
version 1.2.3

Code under GNU General Public License v3 (see `LICENSE` file). <br>
Contact: A. Coulon antoine.coulon@curie.fr – Institut Curie, CNRS. 2022

In [ ]:
###########
dirSrc='/Users/acoulon/res/dev/FISH_analysis/'
###########

from scipy import *
from matplotlib import pyplot as plt
import os, sys
sys.path.append(dirSrc); import locFiles as lf
import pickle

dataSetName=os.path.basename(os.getcwd())
print("\nDataset: %s"%lf.toBold(dataSetName))

## Load data and make list of loci

In [ ]:
#### Load data ####

CCMs=lf.imp.load_source('m', 'chromaticCorr_20210104.py').chromaticCorrMatrices['microscope2_60x_dz0.3um']

lData,cellIds=lf.filterLoc2(
    (lf.loadLoc2('out/'+dataSetName+'_FAM.loc2',    name='Locus',   unit='a.u.',             cutoffLow=3.8e5, chromaticCorrM=CCMs['GFP => o5-Q570'],       color='c'),
     lf.loadLoc2('out/'+dataSetName+'_CAL610.loc2', name='FOS',     unit='RNAs', norm=4.0e4, cutoffLow=.6,    chromaticCorrM=CCMs['o5-CAL610 => o5-Q570'], color='orange'),
     lf.loadLoc2('out/'+dataSetName+'_Q670.loc2',   name='JDP2',    unit='RNAs', norm=3.5e4, cutoffLow=.7,    chromaticCorrM=CCMs['o5-Q670 => o5-Q570'],   color='r'),
     lf.loadLoc2('out/'+dataSetName+'_Q570.loc2',   name='BATF',    unit='RNAs', norm=5.0e4, cutoffLow=.6,    chromaticCorrM=CCMs['o5-Q570 => o5-Q570'],   color='g'),
     lf.loadLoc2('out/'+dataSetName+'_AF750.loc2',  name='coreDel', unit='a.u.',                              chromaticCorrM=CCMs['o5-AF750 => o5-Q570'],  color='k')),
    incl='out/'+dataSetName+'_segmented_nuclei_curated.cellIds',
    excl='out/'+dataSetName+'_segmented_nuclei_excluded.cellIds'
)

mcLocus=lf.mcLoc(lData, cellIds,
                 voxelSize=r_[0.1075, 0.1075, 0.3],
                 nuclei_csv='out/'+dataSetName+'_segmented_nuclei.csv');


<a id='add_neg_shuff_BACK'></a>
__OPTIONAL:__ If using this dataset as a WT control for the deletion spots, go to section "[Add 'negative shuffled' channel](#add_neg_shuff)" and then come back here.

In [ ]:
## Calculate pairwise distances
mcLocus.calcDist([[0,0],[0,1],[0,2],[0,3],[0,4],]); #[0,5]]);


In [ ]:
#### Choose distance threshold ####
dThr=1.3
###

mcLocus.showDist(0,0,dThr=dThr)
mcLocus.showDist(0,1,dThr=dThr)
mcLocus.showDist(0,2,dThr=dThr)
mcLocus.showDist(0,3,dThr=dThr)
mcLocus.showDist(0,4,dThr=dThr)


In [ ]:
## Assign spots in all channels to a locus spot
mcLocus.listLociInCells(0,dThr=dThr,method='brightest')


-----

## Count unpaired spots (e.g. mRNAs) in cells
If your dataset includes smRNA FISH channels, use this to calculate the number of (non nascent) mRNA in each cell.

In [ ]:
#### Number of unpaired spots ####
cellIdsOfUnpairedSpots=[mcLocus.listSpots_pairedAndUnpaired(c)[3][:,-1].astype(int)
                                                                 for c in range(mcLocus.nCh)] 
mcLocus.spotsInCells_unpaired={cId: [int((cId==a).sum()) for a in cellIdsOfUnpairedSpots]
                                                   for cId,ll in mcLocus.lociInCells.items()}

#### Number of spots that are far from a locus ####
cellIdsOfFarSpots=[mcLocus.listSpots_nearAndFar(cRef=0,c=c,dThr=dThr)[3][:,-1].astype(int)
                                                               for c in range(mcLocus.nCh)]
mcLocus.spotsInCells_far={cId: [int((cId==a).sum()) for a in cellIdsOfFarSpots]
                                                   for cId,ll in mcLocus.lociInCells.items()}


----

## Save `mcLoc` object to be used in other notebooks

In [ ]:
## Save object using pickle
pickle.dump(mcLocus, open('out/'+dataSetName+'_mcLoc.pickle', 'wb'))


----
----

## Misc plots

In [ ]:
#### Number of loci per cell ####

nbLociPerCell=[len(ll) for cId,ll in mcLocus.lociInCells.items()]

#dThrBwLoci=1.
#locusSpotsInCells={cId:list(set(dist[0,0][:,3].astype(int)) - set((dist[0,0][dist[0,0][:,0]<dThrBwLoci,3]).astype(int))) for cId,dist in mcLocus.distInCells.items()}
#nbLociPerCell=[len(ll) for cId,ll in locusSpotsInCells.items()]

plt.figure()
plt.hist(nbLociPerCell,bins=r_[:30]-.5)
plt.xlabel('Number of loci per cell'); plt.ylabel('Counts')
plt.xlim(-.5,16.5)
plt.show()

In [ ]:
toPlot=array([[cId, len(mcLocus.lociInCells[cId]), area, mean-min, std, a, b]
                  for cId,area,mean,min,std,a,b in mcLocus.nuclei_csv.loc[:,['cellIds','Area','Mean','Min','StdDev','Skew','Kurt']].to_numpy()
                      if cId in mcLocus.lociInCells])
#toPlot=toPlot[toPlot[:,1]<=8]

plt.figure()
plt.scatter(toPlot[:,1]+(rand(toPlot.shape[0])-.5)*.8,toPlot[:,2]*prod(mcLocus.voxelSize[:2]),alpha=.1,s=15)
plt.xlabel('Number of alleles'); plt.ylabel('Nuclear area (µm$^2$)');
plt.xlim(0,20);
plt.ylim(0,400);
plt.show()


In [ ]:
for d in mcLocus.lData[:-1]:
    print(lf.toBold(d.name))
    d.showFrameAndPositionBiases()
    

In [ ]:
#### Instensity distribution of spots that are –or not– associated with a locus spot

### Choose channel
c=4;
###

plt.figure(); plt.title(mcLocus.lData[c].name)

plt.hist(mcLocus.lData[c][:,0],                              label='All',      bins=mcLocus.lData[c].binsLog, histtype='step', color='gray')
plt.hist(mcLocus.listSpots_pairedAndUnpaired(c)[2][:,0],     label='Paired',   bins=mcLocus.lData[c].binsLog, histtype='step', color='C1')
#plt.hist(mcLocus.listSpots_pairedAndUnpaired(c)[3][:,0],     label='Unpaired', bins=mcLocus.lData[c].binsLog, histtype='step', color='C0', ls=':')
#plt.hist(mcLocus.listSpots_nearAndFar(0,c,dThr=1.3)[2][:,0], label='Near',     bins=mcLocus.lData[c].binsLog, histtype='step', color='C1', ls=':')
plt.hist(mcLocus.listSpots_nearAndFar(0,c,dThr=1.3)[3][:,0], label='Far',      bins=mcLocus.lData[c].binsLog, histtype='step', color='C0')

plt.xlabel('Fluo. intensity (%s)'%(mcLocus.lData[c].unit)); plt.ylabel('Counts'); plt.legend()
plt.xscale('log');

### Adjust plot
plt.yscale('log');
#plt.xlim(.5,None)
###
plt.show()

In [ ]:
#### Separation plot for a single channel

### Choose channel
c=4;
###

d=mcLocus.lData[c]
#d=lf.loadLoc2(mcLocus.lData[c].fn, name=mcLocus.lData[c].name, color=mcLocus.lData[c].color)

lf.evalSingleFluoDistr(d);


In [ ]:
#### Separation plot for known positive and negative distributions

lf.evalPositiveAndNegativeFluoDistr(mcLocus.listSpots_pairedAndUnpaired(4)[2],
                                    mcLocus.listSpots_pairedAndUnpaired(5)[2],
                                    fracSignalSpots=0.4);


In [ ]:
[cId for cId,ll in mcLocus.lociInCells.items()
 if 4<=len(ll)<=6
 and 4<=sum(array(ll)[1:4,0]>0)<=6
 and 4<=mcLocus.spotsInCells_far[cId][1]<=10
 and 4<=mcLocus.spotsInCells_far[cId][2]<=10
 and 4<=mcLocus.spotsInCells_far[cId][3]<=10]


----
----

## Add 'negative shuffled' channel<a id='add_neg_shuff'></a>

In [ ]:
#### Choose distance threshold ####
dThr=1.3
###

mcLocus.calcDist([[0,0],[0,4],[4,4]]);

mcLocus.showDist(0,0,dThr=dThr)
mcLocus.showDist(0,4,dThr=dThr)
mcLocus.showDist(4,4,dThr=0.25)


In [ ]:
#### Pair deletion spots with loci ####

mcLocus.listLociInCells(cRef=0,lCh=[4],dThr=dThr,method='brightest')


In [ ]:
#### Make 'negative shuffled' channel from the deletion channel ####

spots_unpaired=mcLocus.listSpots_pairedAndUnpaired(c=4)[3]
spots_far     =mcLocus.listSpots_nearAndFar(cRef=0,c=4,dThr=1.3)[3]

negShuff=mcLocus.lData[-1].makecopy(); negShuff.name+=' neg. shuff.'; negShuff.color='gray'
negShuff.resize(spots_unpaired.shape,refcheck=False)
negShuff[:,1:]=spots_unpaired[:,1:]
negShuff[:,0] =spots_far[random.randint(0,spots_far.shape[0],spots_unpaired.shape[0]),0]
negShuff.show()

mcLocus=lf.mcLoc(mcLocus.lData+[negShuff], mcLocus.cellIds, voxelSize=mcLocus.voxelSize, nuclei_csv=mcLocus.nuclei_csv);


[Go back](#add_neg_shuff_BACK)

----
----

## Inspect localizations on images with `inspectLoc()`

In [ ]:

lf.inspectLoc(cellId=[2,3,4,34],loc=[mcLocus.listSpots_pairedAndUnpaired(c=2)[2],
                                     mcLocus.listSpots_nearAndFar(cRef=0,c=2,dThr=1.3)[3]],
                                                                          locVal=0,maxProj=1,
              rawCCMs=[CCMs['o5-AF750 => o5-Q570'], CCMs['o5-Q670 => o5-Q570'], CCMs['o5-CAL610 => o5-Q570'], CCMs['o5-Q570 => o5-Q570'], CCMs['GFP => o5-Q570'], CCMs['DAPI => o5-Q570']])


In [ ]:
lf.inspectLoc(cellId=[10,15,16], loc=[mcLocus.lData[0]], locVal=0, maxProj=0,
              rawCCMs=[CCMs['o5-AF750 => o5-Q570'], CCMs['o5-Q670 => o5-Q570'], CCMs['o5-CAL610 => o5-Q570'], CCMs['o5-Q570 => o5-Q570'], CCMs['GFP => o5-Q570'], CCMs['DAPI => o5-Q570']])


In [ ]:
excl=[103,105,208,228,271,340,344,387,442,483,496,585,675,754,873,921,1005,1053,1078,1120,1185,1339,1383,1398,1475,1477,1512,1525,1528,1534,1542,1660,1680,1735,1787,1833,1894,1908,1913,2016,2050,2068,2075,2085,2162,2164,2199,2211,2223,2234,2254,2290,2394,2675,2704,2711,2745,2757,2771,2791,2833,2864,2890,2966,3028,3069,3332,3375,3413,3471,3567,3574,3652,3666,3680,3765,3807,3917,3957,3998,4035,4230,4279,4296,4319,4377,4394,4404,4435,4540,4558,4679,4745,4803]+[97,128,146,212,432,469,722,823,1221,1493,1585,1586,1589,1751,1839,1868,1907,1932,1953,2192,2234,2384,2435,2513,2704,2711,2966,3183,3389,3574,3694,3728,3730,3740,3744,3748,3807,3925,4369,4391,4600]+[39,45,54,135,172,208,259,269,271,297,340,359,453,459,469,494,537,618,720,771,777,780,781,873,960,1108,1120,1187,1206,1212,1217,1267,1292,1417,1451,1477,1481,1488,1491,1566,1582,1590,1617,1643,1671,1689,1744,1753,1781,1790,1798,1828,1854,1883,1913,2016,2082,2085,2177,2218,2234,2275,2299,2478,2518,2525,2527,2612,2640,2658,2673,2683,2704,2711,2736,2762,2782,2785,2786,2791,2812,2845,2854,2875,2966,3019,3046,3064,3108,3239,3265,3358,3395,3420,3441,3481,3574,3575,3588,3628,3679,3680,3714,3747,3807,3859,3957,4075,4083,4120,4147,4180,4206,4253,4267,4279,4297,4299,4302,4307,4312,4314,4319,4321,4323,4324,4325,4339,4342,4349,4357,4358,4362,4372,4374,4376,4377,4387,4391,4404,4405,4410,4469,4487,4545,4555,4556,4585,4644,4691,4718,4799]
savetxt('out/'+dataSetName+'_segmented_nuclei_excluded.cellIds',excl,fmt='%i')